# 프로젝트 구조
```bash
code
├── base_line_codes
│   ├── Dataset.py
│   ├── evaluation.py
│   ├── test.py
│   ├── label_maker.py
│   ├── train.py
│   └── model.py
├── logs
├── parameters
├── project.toml
├── project.lock
├── train_data.csv
└── main.ipynb
``` 
<span style="color: red;">위 구조대로 파일을 구성해주세요</span> <br>
<span style="color: red;">parameters, project.lock, logs, train_data.csv 는 자동으로 생성됩니다</span>

# 필요한 라이브러리 설치 및 버전 맞추기

In [1]:
!pip install ruamel_yaml
!pip install urllib3
!pip install poetry --use-feature=2020-resolver
#!pip install tensorboard

In [2]:
import base_line_codes.train
import base_line_codes.label_maker
import tensorboard
import torch
from torch.utils.tensorboard import SummaryWriter
import gc
import os

# 경로 지정 및 캐시 초기화
#### tensor board log 위치 : /logs

In [3]:
writer = SummaryWriter('logs/')

# 캐시 초기화
gc.collect()
torch.cuda.empty_cache()

# train data labeling
### label
```markdown
* mask
    * mask : 0
    * incorrect : 1
    * normal : 2
* gender
    * female : 0
    * male : 1
* age
    * ~30 : 0
    * 30~60 : 1
    * 60~ : 2
```

In [4]:
base_line_codes.label_maker.label_maker()

데이터 수 : 18900


# train
### parameter and default value
```python
train_model(
    LEARNING_RATE=1e-3,
    EPOCH=5,
    BATCH_SIZE=64,
    PRETRAINED=False,   (True, False)
    LABEL='multi',      (multy, mask, gender, age)
    VAL_SIZE=0.2        (0 ~ 1) validation data 의 비율
)
```

### mask_model, age_model, gender_model 학습

In [5]:
mask_model = base_line_codes.train.train_model(LABEL='mask')
age_model = base_line_codes.train.train_model(LABEL='age')
gender_model = base_line_codes.train.train_model(LABEL='gender')

--------------------train setting start--------------------
|---set model
|---set loss
|---set optimizer
|---set dataset
|---set dataloader
---------------------train setting end---------------------

train start
Iter [  0/237] | Train Loss 1.1421 | Train Acc 0.0938
Iter [ 20/237] | Train Loss 0.8397 | Train Acc 0.6719
Iter [ 40/237] | Train Loss 0.9145 | Train Acc 0.6562
Iter [ 60/237] | Train Loss 0.5657 | Train Acc 0.7969
Iter [ 80/237] | Train Loss 0.6844 | Train Acc 0.7500
Iter [100/237] | Train Loss 0.5343 | Train Acc 0.8281
Iter [120/237] | Train Loss 0.7128 | Train Acc 0.7500
Iter [140/237] | Train Loss 1.0575 | Train Acc 0.5781
Iter [160/237] | Train Loss 0.9046 | Train Acc 0.6250
Iter [180/237] | Train Loss 0.5694 | Train Acc 0.7812
Iter [200/237] | Train Loss 0.4371 | Train Acc 0.8281
Iter [220/237] | Train Loss 0.2636 | Train Acc 0.9375
label acc : [0.966, 0.024, 0.321], total_acc : 0.741, f1_score : 0.439
VALIDATION
Valid Loss 0.3965 | Valid Acc 0.8381
([0.979, 0.085, 0.91

# 텐서보드 / 스킵

In [6]:
'''
%load_ext tensorboard
# if not 령태:
%tensorboard --logdir=logs --host=0.0.0.0 --port=30001
# else:
#%tensorboard --logdir=./logs --host=0.0.0.0 --port=30005
'''

'\n%load_ext tensorboard\n# if not 령태:\n%tensorboard --logdir=logs --host=0.0.0.0 --port=30001\n# else:\n#%tensorboard --logdir=./logs --host=0.0.0.0 --port=30005\n'

# valid_class

In [7]:
base_line_codes.train.classify_label(BATCH_SIZE=64, VAL_SIZE=0.2, MASK_MODEL=mask_model, GENDER_MODEL=gender_model, AGE_MODEL=age_model)

--------------------train setting start--------------------
|---set model
|---set dataset
|---set dataloader
---------------------train setting end---------------------


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>